In [1]:
import sys
import rospy
import rospkg
import moveit_commander
import tf
import os 
import numpy as np 
import copy
import time

from geometry_msgs.msg import (
    PoseStamped,
    Pose,
    Point,
    Quaternion,
)
from gazebo_msgs.srv import (SpawnModel,DeleteModel)
from std_msgs.msg import Header
from geometry_msgs.msg import (PoseStamped,Pose,Point,Quaternion)
from trajectory_msgs.msg import JointTrajectory
from trajectory_msgs.msg import JointTrajectoryPoint

print("Packages Loaded.")
rospy.init_node("object_spawn")

Packages Loaded.


### Set Moveit 

In [2]:
moveit_commander.roscpp_initialize(sys.argv)
robot         = moveit_commander.RobotCommander()
scene         = moveit_commander.PlanningSceneInterface()
group_name    = robot.get_group_names()
print("Moveit control group name:%s"%group_name)
arm_group     = moveit_commander.MoveGroupCommander("arm")
gripper_group = moveit_commander.MoveGroupCommander("gripper")

[ INFO] [1651471740.006674248]: Loading robot model 'rilab_robot'...
[ INFO] [1651471740.007151785]: No root/virtual joint specified in SRDF. Assuming fixed joint
Moveit control group name:['arm', 'gripper']
[ INFO] [1651471741.251511874, 37.160000000]: Ready to take commands for planning group arm.
[ INFO] [1651471741.723870425, 37.632000000]: Ready to take commands for planning group gripper.


## Spawn-Delete Object 

In [ ]:
def spawn_gazebo_model(model_path, model_name, model_pose, reference_frame="world"):
  """
  Spawn model in gazebo
  """
  model_xml = ''
  with open(model_path, "r") as model_file:
    model_xml = model_file.read().replace('\n', '')
  rospy.wait_for_service('/gazebo/spawn_sdf_model')
  try:
    spawn_sdf = rospy.ServiceProxy('/gazebo/spawn_sdf_model', SpawnModel)
    resp_sdf= spawn_sdf(model_name, model_xml, "/", model_pose, reference_frame)
  except rospy.ServiceException:
    rospy.logerr("Spawn SDF service call failed")

def delete_gazebo_model(models):
  """
  Delete model in gazebo
  """
  try:
    delete_model = rospy.ServiceProxy('/gazebo/delete_model', DeleteModel)
    for a_model in models:
      resp_delete = delete_model(a_model)
  except rospy.ServiceException:
    rospy.loginfo("Delete Model service call failed")


### Set Parameters 

In [3]:
# Parameters 
table_height    = 0.79
# Cylinder 
cylinder_height = 0.2
cylinder_radius = 0.03
cylinder_pos_x  = 0.97 
cylinder_pos_y  = 0 
cylinder_pos_z  = table_height+cylinder_height/2.
# Box 
box_height      = 0.06
box_x           = 0.06 
box_y           = 0.06
box_pos_x       = 0.97 
box_pos_y       = 0 
box_pos_z       = table_height+box_height/2.

In [4]:
""" RViz """
# Cylinder 
cylinder_name = "cylinder"
cylinder_pos  = PoseStamped()
cylinder_pos.header.frame_id    = robot.get_planning_frame()
cylinder_pos.pose.position.x    = cylinder_pos_x
cylinder_pos.pose.position.y    = cylinder_pos_y
cylinder_pos.pose.position.z    = cylinder_pos_z
cylinder_pos.pose.orientation.w = 1.0  
# Box 
box_name = "box"
box_pos  = PoseStamped()
box_pos.header.frame_id    = robot.get_planning_frame()
box_pos.pose.position.x    = box_pos_x
box_pos.pose.position.y    = box_pos_y
box_pos.pose.position.z    = box_pos_z
box_pos.pose.orientation.w = 1.0  

""" GAZEBO """
rospack = rospkg.RosPack()
pack_path = rospack.get_path('ir_gazebo')
object_path = pack_path+os.sep+'urdf'+os.sep+'object'+os.sep+'cylinder.sdf'
object_name = cylinder_name
object_pose = Pose(position=Point(x=cylinder_pos_x , y=cylinder_pos_y, z=cylinder_pos_z))


## Trajectory 

In [5]:
arm_joint_names = ['shoulder_pan_joint','shoulder_lift_joint','elbow_joint', 
                'wrist_1_joint', 'wrist_2_joint','wrist_3_joint']
gripper_joint_names = ['gripper_finger1_joint']
rate    = rospy.Rate(50)
arm_pub = rospy.Publisher('/arm_controller/command', JointTrajectory, queue_size=10)
gripper_pub = rospy.Publisher('/gripper_controller/command', JointTrajectory, queue_size=10)

""" Load joint values """
# load_data       = np.load("data/sample_traj.npz")
load_data_pick  = np.load("data/sample_traj_pick.npz")
load_data_place = np.load("data/sample_traj_place.npz")
load_data_home  = np.load("data/sample_traj_home.npz")

joints_seq_pick  = load_data_pick["joint"]
joints_seq_place = load_data_place["joint"]
joints_seq_home  = load_data_home["joint"]

touch_links = ["gripper_finger1_finger_link", "gripper_finger2_finger_link"]

def move_gripper(joint):
    global gripper_joint_names, gripper_pub
    """ Open- Close Gripper """
    gripper        = JointTrajectory()
    gripper_value  = JointTrajectoryPoint()
    gripper.header = Header()
    gripper.joint_names = gripper_joint_names
    gripper_value.positions = [joint] # Open pose
    gripper_value.time_from_start = rospy.Duration.from_sec(0.01)
    gripper.points.append(gripper_value)
    gripper_pub.publish(gripper)


def move_arm(joint_seq): 
    Flag = True 
    global arm_joint_names, rate, arm_pub
    while Flag:
        """ Move to pick an object """
        for idx, joints in enumerate(joint_seq):
            joint = joints.reshape([6,])
            arm = JointTrajectory()
            arm_value = JointTrajectoryPoint()
            arm.header = Header()
            arm.joint_names= arm_joint_names
            arm_value.positions       = joint
            arm_value.time_from_start = rospy.Duration.from_sec(0.01)
            arm.points.append(arm_value)
            arm_pub.publish(arm)
            rate.sleep()    
            if idx+1 == len(joint_seq): 
                Flag = False 

### Start

In [6]:
# Move to start pose 
move_arm(joints_seq_home)
# If you want to add a cylinder on RViz 
scene.add_cylinder(name=object_name, height=cylinder_height, radius=cylinder_radius, pose=cylinder_pos)
# If you want to add a cylinder on GAZEBO 
spawn_gazebo_model(object_path, object_name, object_pose)

### Plan

In [8]:
# Move to pick an object 
move_arm(joints_seq_pick)
time.sleep(1)
# Open gripper 
move_gripper(0.8)
# Attach the object 
gripper_group.attach_object(object_name, "gripper_base_link", touch_links=touch_links)
time.sleep(1)
# Close gripper 
move_gripper(0.3)
# Move to place the object 
move_arm(joints_seq_place)
# Open gripper 
move_gripper(0.8)


### Initialize

In [9]:
# Delete the object on GAZEBO 
delete_gazebo_model([object_name])
# Detach the object from the gripper 
gripper_group.detach_object(object_name)
# Delte the object on RViz 
scene.remove_world_object(object_name)
# Move to initial pose 
move_arm(joints_seq_home)
# Close gripper 
move_gripper(0.)
print("Ready to start")

Ready to start
